### Import

Dataset : https://www.kaggle.com/datasets/humansintheloop/semantic-segmentation-of-aerial-imagery

In [2]:
!pip install patchify

In [11]:
!pip install segmentation_models_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=ff837d2148a69a59b8bcb4054709e139191ab02dbea80fd1c2ef37fd36d2fb20
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60966 sha256=cd47b43f8e5f3b1126eaefaad02610f3f3a8eb353501020e9a0b16f7311f4874
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch pretrainedmodels


In [3]:
import os
import re
from pathlib import Path
import numpy as np
from patchify import patchify
from PIL import Image

In [13]:
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
import segmentation_models_pytorch as smp
import seaborn as sns
import matplotlib.pyplot as plt

### Create patches

In [5]:
def create_folder():
    FOLDERS = ['train','val','test']
    
    for folder in FOLDERS:
        if not os.path.exists(folder):
            folder_imgs = f"{folder}/images"
            folder_msks = f"{folder}/masks"
            os.makedirs(folder_imgs) if not os.path.exists(folder_imgs) else print('folder already exists')
            os.makedirs(folder_msks) if not os.path.exists(folder_msks) else print('folder already exists')

In [6]:
create_folder()

### Create Patches

In [7]:
def create_patches(src, dest_path):
    path_split = os.path.split(src)
    tile_num = re.findall(r'\d+', path_split[0])
    image = Image.open(src)
    image = np.asarray(image)
    if len(image.shape) > 2:
        patches = patchify(image, (320, 320, 3), step = 300)
        file_name_wo_ext = Path(src).stem
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):
                patch = patches[i, j, 0]
                num = i * patches.shape[1] + j
                patch = Image.fromarray(patch)
                patch.save(f"{dest_path}/{file_name_wo_ext}_tile_{tile_num}_patch_{num}.png")
                

In [8]:
for path_name, _, file_name in os.walk('/kaggle/input/semantic-segmentation-of-aerial-imagery/Semantic segmentation dataset'):
    for f in file_name:
        print(f)
        if f != 'classes.json':
            path_split = os.path.split(path_name)
            tile_num = re.findall(r'\d+',path_split[0])[0]
            img_type = path_split[1]
            
            if tile_num == '3':
                target_folder_imgs = 'val'
                target_foler_masks = 'val'
            elif tile_num == '1':
                target_folder_imgs = 'test'
                target_foler_masks = 'test'
            elif tile_num in ['4', '5', '6', '7', '8']:
                target_folder_imgs = 'train'
                target_foler_masks = 'train'
                
            # copy all images
            src = os.path.join(path_name, f)
            file_name_wo_ext = Path(src).stem
            
            # Check if file exists in images and masks
            img_file = f"{path_split[0]}/images/{file_name_wo_ext}.jpg"
            mask_file = f"{path_split[0]}/masks/{file_name_wo_ext}.png"
            
            if os.path.exists(img_file) and os.path.exists(mask_file):
                if img_type == 'images':
                    dest = os.path.join(target_folder_imgs, img_type)
                    create_patches(src, dest)
                
                if img_type == 'masks':
                    dest = os.path.join(target_foler_masks, img_type)
                    create_patches(src, dest)                

classes.json
image_part_002.jpg
image_part_006.jpg
image_part_005.jpg
image_part_003.jpg
image_part_004.jpg
image_part_007.jpg
image_part_009.jpg
image_part_008.jpg
image_part_001.jpg
image_part_001.png
image_part_003.png
image_part_006.png
image_part_002.png
image_part_008.png
image_part_007.png
image_part_009.png
image_part_005.png
image_part_004.png
image_part_002.jpg
image_part_006.jpg
image_part_005.jpg
image_part_003.jpg
image_part_004.jpg
image_part_007.jpg
image_part_009.jpg
image_part_008.jpg
image_part_001.jpg
image_part_001.png
image_part_003.png
image_part_006.png
image_part_002.png
image_part_008.png
image_part_007.png
image_part_009.png
image_part_005.png
image_part_004.png
image_part_002.jpg
image_part_006.jpg
image_part_005.jpg
image_part_003.jpg
image_part_004.jpg
image_part_007.jpg
image_part_009.jpg
image_part_008.jpg
image_part_001.jpg
image_part_001.png
image_part_003.png
image_part_006.png
image_part_002.png
image_part_008.png
image_part_007.png
image_part_009.png

### Create Dataset class

In [22]:
class SegmentationDataset(Dataset):
    
    def __init__(self, path_name) -> None:
        super().__init__()
        self.image_names = os.listdir(f"{path_name}/images")
        self.image_paths = [f"{path_name}/images/{i}" for i in self.image_names]
        self.mask_names = os.listdir(f"{path_name}/masks")
        self.mask_paths = [f"{path_name}/images/{i}" for i in self.mask_names]
        
        # filter all images that do not exists in both folder
        self.img_stem = [Path(i).stem for i in self.image_paths]
        self.msk_stem = [Path(i).stem for i in self.mask_paths]
        
        self.img_msk_stem = set(self.img_stem) & set(self.msk_stem)
        
        self.image_paths = [i for i in self.image_paths if (Path(i).stem in self.img_msk_stem)]
    
    def __len__(self):
        return len(self.img_msk_stem)
    
    def convert_mask(self, mask):
        mask[mask == 155] = 0 # unlabelled
        mask[mask == 44] = 1 # building
        mask[mask == 91] = 2 # land
        mask[mask == 171] = 3 # water
        mask[mask == 172] = 4 # road
        mask[mask == 212] = 5 # vegetation
    
    def __getitem__(self, index):
        image = cv2.imread(self.image_paths[index])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        image = image.transpose((2, 0, 1))
        mask = cv2.imread(self.mask_paths[index], 0)
        mask = self.convert_mask(mask)
        return image, mask
        
        

### Modelling

In [14]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [15]:
DEVICE

'cpu'

In [16]:
EPOCHS = 10

In [17]:
BATCH_SIZE = 4

In [23]:
# Create dataloader
train_ds = SegmentationDataset(path_name = 'train')
train_dataloader = DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle = True)
val_ds = SegmentationDataset(path_name = 'val')
val_dataloader = DataLoader(val_ds, batch_size = BATCH_SIZE, shuffle = True)

In [24]:
train_ds[0]

(array([[[101, 100,  98, ..., 193, 191, 187],
         [106, 108, 107, ..., 191, 186, 181],
         [110, 114, 113, ..., 182, 174, 180],
         ...,
         [205, 210, 211, ..., 193, 185, 162],
         [208, 214, 210, ..., 183, 178, 153],
         [206, 207, 202, ..., 193, 173, 137]],
 
        [[ 83,  81,  79, ..., 200, 198, 194],
         [ 87,  88,  87, ..., 198, 193, 188],
         [ 89,  91,  90, ..., 189, 181, 187],
         ...,
         [213, 218, 219, ..., 199, 191, 168],
         [216, 222, 218, ..., 191, 186, 161],
         [214, 215, 210, ..., 203, 181, 147]],
 
        [[ 79,  75,  73, ..., 210, 208, 204],
         [ 81,  81,  80, ..., 208, 203, 198],
         [ 84,  85,  84, ..., 199, 191, 197],
         ...,
         [224, 229, 230, ..., 211, 203, 180],
         [227, 233, 229, ..., 204, 199, 174],
         [225, 226, 221, ..., 215, 194, 159]]], dtype=uint8),
 None)